In [1]:
# import libraries
import mne, os, pickle, glob, sklearn
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt


# setting

In [2]:
# directories
main_dir = '/Users/cl5564/Dropbox/Analysis/NTULingAmb/'
os.chdir(main_dir) # change directory
meg_dir = os.path.join(main_dir, '02_meg')  # meg directory
mri_dir = os.path.join(main_dir, '03_mri')  # mri directory


# load data

In [3]:
sid = 25 # subject ID
raw_fname = os.path.join(meg_dir, '%.3d', '%.3d-raw.fif') %(sid, sid)
raw = mne.io.read_raw_fif(raw_fname, preload=True)
raw.pick_types(meg=True, stim=True)
raw.info
#raw.plot_sensors(show_names=False)


Opening raw data file /Users/cl5564/Dropbox/Analysis/NTULingAmb/02_meg/025/025-raw.fif...
    Range : 0 ... 939999 =      0.000 ...   939.999 secs
Ready.
Reading 0 ... 939999  =      0.000 ...   939.999 secs...


<Info | 13 non-empty values
 bads: []
 ch_names: MEG 001, MEG 002, MEG 003, MEG 004, MEG 005, MEG 006, MEG 007, ...
 chs: 157 MAG, 1 STIM
 custom_ref_applied: False
 description: Academia Sinica/Institute of Linguistics//Magnetoencephalograph ...
 dev_head_t: MEG device -> head transform
 dig: 435 items (3 Cardinal, 5 HPI, 427 Extra)
 file_id: 4 items (dict)
 highpass: 0.0 Hz
 kit_system_id: 261 (Academia Sinica, 2008-)
 lowpass: 100.0 Hz
 meas_date: 2022-03-31 18:37:05 UTC
 meas_id: 4 items (dict)
 nchan: 158
 projs: []
 sfreq: 1000.0 Hz
>

# plotting and annotating

In [ ]:
# events = mne.find_events(raw, stim_channel='STI 014') # find events
# raw.plot(events=events, event_color='orange', n_channels=70, duration=50)


In [ ]:
# annot_fname = os.path.join(meg_dir, '%.3d', '%.3d_annot.csv') %(sid, sid)
# print('save annotations')
# raw.annotations.save(annot_fname)
# print(raw.annotations)


# filtering

In [5]:
# bad channels
if sid == 18 or sid == 20:
    bads = ['MEG 033','MEG 037','MEG 034', 'MEG 035','MEG 039'] 
else:
    bads = []

# filtering
raw_filt = raw.filter(0, 30, phase='zero-double')

# update bad channel info
raw_filt.info['bads'] = bads

# plotting data
events = mne.find_events(raw, stim_channel='STI 014') # find events
raw_filt.plot_psd(fmax=50)
raw_filt.plot(events=events, event_color='orange', n_channels=70, duration=50)


Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-12 dB cutoff frequency: 33.75 Hz)
- Filter length: 441 samples (0.441 sec)

384 events found
Event IDs: [  1   2   4   8  32 128]
Effective window size : 2.048 (s)
Channels marked as bad: none


<MNEBrowseFigure size 5120x2604 with 4 Axes>

Channels marked as bad: none


# annotating movement artifacts as bad spans

annotating bad spans of data with movement artifacts before ICA <br>
https://mne.tools/stable/auto_tutorials/raw/30_annotate_raw.html

In [6]:
annot_fname = os.path.join(meg_dir, '%.3d', '%.3d_annot.csv') %(sid, sid)

if os.path.exists(annot_fname):
    print('read annotations')
    annot_badspan = mne.read_annotations(annot_fname)
    raw_filt.set_annotations(annot_badspan, emit_warning=False)
    print(raw_filt.annotations)
else:
    print('no annotation file!')
    

read annotations
<Annotations | 2 segments: BAD_ (2)>


# epoching (before ICA)

In [7]:
# define variables
if sid > 20: 
    event_id = {'ambiguous-dominant/193': 2, 'ambiguous-dominant/195': 8,
                'ambiguous-subordinate/194': 1, 'ambiguous-subordinate/192': 4,
                'unambiguous/197': 32, 'unambiguous/199': 128}
else:
    event_id = {'ambiguous-dominant/193': 193, 'ambiguous-dominant/195': 195,
                'ambiguous-subordinate/194': 194, 'ambiguous-subordinate/192': 192,
                'unambiguous/197': 197, 'unambiguous/199': 199}

tmin = -0.1
tmax = 1.0
baseline = (None, 0)  # tmin~0 , -0.1s~0s # none = start from tmin.
picks = mne.pick_types(raw_filt.info, meg=True, stim=False)
events = mne.find_events(raw_filt, stim_channel='STI 014') # find events

# selected events
events = events[range(1, 383, 3)]
print(events.shape)

# Rejection parameters based on peak-to-peak amplitude.
# diff chans (meg/grad) have diff criteria. Check before use.
reject = dict(mag=3e-12) # T (magnetometers)

# epoching
epochs = mne.Epochs(raw_filt, events=events, event_id=event_id,
                    tmin=tmin, tmax=tmax, baseline=baseline,
                    picks=picks, reject=reject, preload=True)


384 events found
Event IDs: [  1   2   4   8  32 128]
(128, 3)
Not setting metadata
Not setting metadata
128 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 128 events and 1101 original time points ...
    Rejecting  epoch based on MAG : ['MEG 029', 'MEG 047', 'MEG 055', 'MEG 092', 'MEG 136', 'MEG 138', 'MEG 139', 'MEG 140', 'MEG 142']
    Rejecting  epoch based on MAG : ['MEG 055', 'MEG 092', 'MEG 140', 'MEG 142']
2 bad epochs dropped


In [8]:
print(epochs.drop_log) #check which trials rejected.
epochs

((), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ('MEG 029', 'MEG 047', 'MEG 055', 'MEG 092', 'MEG 136', 'MEG 138', 'MEG 139', 'MEG 140', 'MEG 142'), (), (), ('MEG 055', 'MEG 092', 'MEG 140', 'MEG 142'), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ())


Number of events,126
Events,ambiguous-dominant/193: 16ambiguous-dominant/195: 16ambiguous-subordinate/192: 16ambiguous-subordinate/194: 15unambiguous/197: 32unambiguous/199: 31
Time range,-0.100 – 1.000 sec
Baseline,-0.100 – 0.000 sec


# ICA

In [9]:
ica = mne.preprocessing.ICA(n_components=15, random_state=97) #max_iter=800
ica.fit(raw_filt, reject_by_annotation=True)
ica.plot_sources(raw_filt)
ica.plot_components()


Fitting ICA to data using 157 channels (please be patient, this may take a while)
Omitting 126089 of 940000 (13.41%) samples, retaining 813911 (86.59%) samples.


<ipython-input-9-317117d5c3e0>:1: DeprecationWarning: Version 0.23 introduced max_iter="auto", setting max_iter=1000 for `fastica` and max_iter=500 for `infomax` and `picard`. The current default of max_iter=200 will be changed to "auto" in version 0.24.
  ica = mne.preprocessing.ICA(n_components=15, random_state=97) #max_iter=800
<ipython-input-9-317117d5c3e0>:2: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(raw_filt, reject_by_annotation=True)


Selecting by number: 15 components
Fitting ICA took 26.9s.
Creating RawArray with float64 data, n_channels=15, n_times=940000
    Range : 0 ... 939999 =      0.000 ...   939.999 secs
Ready.
Channels marked as bad: none


[<MNEFigure size 1950x1462 with 15 Axes>]

In [10]:
print(ica.exclude)
ica.plot_overlay(epochs.average(), exclude=ica.exclude)


[3, 7]
Applying ICA to Evoked instance
    Transforming to ICA space (15 components)
    Zeroing out 2 ICA components
    Projecting back using 157 PCA components


<ipython-input-10-0a5d6990070a>:2: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  ica.plot_overlay(epochs.average(), exclude=ica.exclude)


<Figure size 1280x960 with 1 Axes>

In [11]:
# save ica file
ica_fname = os.path.join(meg_dir, '%.3d', '%.3d-ica.fif') %(sid, sid)
ica.save(ica_fname)
#mne.preprocessing.read_ica(ica_fname)

# save raw file after removing ica
raw_ica_fname = os.path.join(meg_dir, '%.3d', '%.3d_ICA-raw.fif') %(sid, sid)
raw_filt_ica = ica.apply(raw_filt, exclude=ica.exclude)
raw_filt_ica.save(raw_ica_fname, overwrite=True)


Writing ICA solution to /Users/cl5564/Dropbox/Analysis/NTULingAmb/02_meg/025/025-ica.fif...
Applying ICA to Raw instance
    Transforming to ICA space (15 components)
    Zeroing out 2 ICA components
    Projecting back using 157 PCA components
Overwriting existing file.
Writing /Users/cl5564/Dropbox/Analysis/NTULingAmb/02_meg/025/025_ICA-raw.fif
Closing /Users/cl5564/Dropbox/Analysis/NTULingAmb/02_meg/025/025_ICA-raw.fif
[done]


# epoching (after ICA)

In [12]:
# define variables
if sid > 20: 
    event_id = {'ambiguous-dominant/193': 2, 'ambiguous-dominant/195': 8,
                'ambiguous-subordinate/194': 1, 'ambiguous-subordinate/192': 4,
                'unambiguous/197': 32, 'unambiguous/199': 128}
else:
    event_id = {'ambiguous-dominant/193': 193, 'ambiguous-dominant/195': 195,
                'ambiguous-subordinate/194': 194, 'ambiguous-subordinate/192': 192,
                'unambiguous/197': 197, 'unambiguous/199': 199}

tmin = -0.1
tmax = 1.0
baseline = (None, 0)  # tmin~0 , -0.1s~0s # none = start from tmin.
picks = mne.pick_types(raw_filt.info, meg=True, stim=False)
events = mne.find_events(raw_filt, stim_channel='STI 014') # find events

# selected events
events = events[range(1, 383, 3)]
print(events.shape)

# Rejection parameters based on peak-to-peak amplitude.
# diff chans (meg/grad) have diff criteria. Check before use.
reject = dict(mag=3e-12) # T (magnetometers)

# epoching
decim = 1 # 1 for no decimation (downsampling)
epochs = mne.Epochs(raw_filt_ica, events=events, event_id=event_id,
                    tmin=tmin, tmax=tmax, baseline=baseline, decim=decim,
                    picks=picks, reject=reject, preload=True)

# save epochs
epochs_fname = os.path.join(meg_dir, '%.3d', '%.3d-epo.fif') %(sid, sid)
epochs.save(epochs_fname, overwrite=True)


384 events found
Event IDs: [  1   2   4   8  32 128]
(128, 3)
Not setting metadata
Not setting metadata
128 matching events found
Setting baseline interval to [-0.1, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 128 events and 1101 original time points ...
    Rejecting  epoch based on MAG : ['MEG 029', 'MEG 047', 'MEG 055', 'MEG 092', 'MEG 136', 'MEG 138', 'MEG 139', 'MEG 140', 'MEG 142']
    Rejecting  epoch based on MAG : ['MEG 055', 'MEG 092', 'MEG 140', 'MEG 142']
2 bad epochs dropped
Overwriting existing file.


# evoked responses

In [13]:
# evoked responses for each condition
evoked_amb_dom = epochs['ambiguous-dominant'].average()
evoked_amb_dom.comment = 'ambiguous (dominant biasing)'

evoked_amb_sub = epochs['ambiguous-subordinate'].average()
evoked_amb_sub.comment = 'ambiguous (subordinate biasing)'

evoked_unamb = epochs['unambiguous'].average()
evoked_unamb.comment = 'unambiguous'

print(evoked_amb_dom)
print(evoked_amb_sub)
print(evoked_unamb)

# save evoked responses
evoked_fname = os.path.join(meg_dir, '%.3d', '%.3d_ambiguity-ave.fif') %(sid, sid)
mne.write_evokeds(evoked_fname, [evoked_amb_dom, evoked_amb_sub, evoked_unamb])


<Evoked | 'ambiguous (dominant biasing)' (average, N=32), -0.1 – 1 sec, baseline -0.1 – 0 sec, 157 ch, ~1.6 MB>
<Evoked | 'ambiguous (subordinate biasing)' (average, N=31), -0.1 – 1 sec, baseline -0.1 – 0 sec, 157 ch, ~1.6 MB>
<Evoked | 'unambiguous' (average, N=63), -0.1 – 1 sec, baseline -0.1 – 0 sec, 157 ch, ~1.6 MB>


# plotting ERF at each sensor

In [ ]:
evoked_conds = [evoked_amb_dom, evoked_amb_sub, evoked_unamb]
colors = 'red', 'blue', 'green'
mne.viz.plot_evoked_topo(evoked_conds, color=colors)


# butterfly plots for each condition

In [14]:
# make directory for saving figures
evoked_dir = os.path.join(meg_dir, '%.3d', 'evoked') %(sid)
if not os.path.exists(evoked_dir):
    os.makedirs(evoked_dir)

evoked_conds = ['evoked_amb_dom', 'evoked_amb_sub', 'evoked_unamb']
for evk_name in evoked_conds:
    evoked_fname = os.path.join(evoked_dir, '%.3d_%s.png') %(sid, evk_name)
    evoked_temp = eval(evk_name)
    evoked_plot = evoked_temp.plot_joint(
        ts_args = dict(gfp=True, ylim=dict(mag=[-350, 350]), time_unit='ms'),
        topomap_args = dict(vmin=-200, vmax=200, time_unit='ms'),
        title = 'Magnetometers (157 channels) \n %s'%(evoked_temp.comment), show = False)
    
    evoked_plot.savefig(evoked_fname)

plt.close('all')


No projector specified for this dataset. Please consider the method self.add_proj.
No projector specified for this dataset. Please consider the method self.add_proj.
No projector specified for this dataset. Please consider the method self.add_proj.
